In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
from tqdm import trange
from sklearn.model_selection import train_test_split

from src.data.ucmerced_dataset import UcMercedDataset
from src.settings import DATA_DIRECTORY

In [3]:
dataset = UcMercedDataset(DATA_DIRECTORY)

In [13]:
x = np.empty(shape=(dataset.__len__(), 256, 256, 3))
y = np.empty(shape=(dataset.__len__(), ), dtype=np.int)

for idx in trange(dataset.__len__()):
    item = dataset.__getitem__(idx)

    x[idx] = item['a']
    y[idx] = item['a_y']

100%|██████████| 2056/2056 [00:14<00:00, 141.06it/s]


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)